In [1]:
# !pip install open3d
import open3d as o3d
import numpy as np
import sys

sys.path.append('./data/')
from camera import *
from dataloader import *

sys.path.append('./algorithm/')
from measurement import *
from pose import *
from update import *
from prediction import *

from pipeline import *

import matplotlib.pyplot as plt
import time

In [3]:
%cd eval

/Users/erbao/Study/cs231a/project/KinectFusion/eval


In [7]:
%%python evaluate_ate.py ../data/dataset/rgbd_dataset_freiburg3_cabinet/groundtruth.txt ../results/trajectory.txt --plot ../results/ate.png

UsageError: %%python is a cell magic, but the cell body is empty.


In [ ]:
vbg = o3d.t.geometry.VoxelBlockGrid.load('./results/TSDF.npz')
visualize_vbg_o3d(vbg, 180, 0, 0)

PointCloud on CPU:0 [533549 points (Float32)].
Attributes: normals (dtype = Float32, shape = {533549, 3}).
FEngine (64 bits) created at 0x7fc46b800000 (threading is enabled)
FEngine resolved backend: OpenGL


[error] GLFW error: Cocoa: Failed to find service port for display
